* Ray RLlib 노트북

필요 패키지 삽입

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

from horcrux_terrain_v1.envs import SandWorld
from horcrux_terrain_v1.envs import PlaneWorld
from horcrux_terrain_v1.envs import PlaneCGWorld

import ray
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig

from ray.tune.registry import register_env


c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Ray 실행 (Warning 관련 무시 키워드)

In [2]:
ray.init(runtime_env={"env_vars": {"PYTHONWARNINGS": "ignore::DeprecationWarning"}})

2024-09-26 08:25:42,811	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.5
Ray version:,2.36.1
Dashboard:,http://127.0.0.1:8265


Gym -> Rllib Env 등록

In [3]:
env_config = {
    "forward_reward_weight": 4,
    "side_cost_weight": 2,
    "unhealthy_max_steps": 75,
    "healthy_roll_range": (-45,45),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.05,
    "termination_reward": 0,
}

# Plane CG
register_env("plane-cg", lambda config: PlaneCGWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 terminating_roll_range=env_config["terminating_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 termination_reward=env_config["termination_reward"]))


학습 알고리즘 설정

In [4]:
config = SACConfig()
# Activate new API stack. -> 구려서 안씀.
config.api_stack(
    enable_rl_module_and_learner=False,
    enable_env_runner_and_connector_v2=False,
)
# config.environment("sand-v1")
config.environment("plane-cg")
config.framework("torch")
total_workers = 12
config.resources(num_gpus=1,num_cpus_per_worker=1, num_gpus_per_worker= 1/(total_workers+1))
config.rollouts(num_rollout_workers=total_workers)
config.training(
    gamma=0.9, 
    # kl_coeff=0.3, 
    replay_buffer_config = {
            "_enable_replay_buffer_api": True,
            "type": "MultiAgentPrioritizedReplayBuffer",
            "capacity": int(1e6),
            # If True prioritized replay buffer will be used.
            "prioritized_replay": False,
            "prioritized_replay_alpha": 0.6,
            "prioritized_replay_beta": 0.4,
            "prioritized_replay_eps": 1e-6,
            # Whether to compute priorities already on the remote worker side.
            "worker_side_prioritization": False,
        },

    # See model catalog for more options.
    # https://docs.ray.io/en/latest/rllib/rllib-models.html
    q_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512],
            "fcnet_activation": "relu",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define custom Q-model(s).
            "custom_model_config": {},
        },
    policy_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512],
            "fcnet_activation": "relu",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define a custom policy model.
            "custom_model_config": {},
        },
    train_batch_size_per_learner = 8192,
    num_steps_sampled_before_learning_starts = 3000,

)
config.evaluation(evaluation_interval=100)

# # See model catalog for more options.
# # https://docs.ray.io/en/latest/rllib/rllib-models.html
# # config.model["fcnet_hiddens"] = [512, 512, 512, 512, 512]
# config.model["uses_new_env_runners"] = True
# config.model["fcnet_hiddens"] = [1024, 1024, 1024, 1024, 1024]
# config.model["use_lstm"] = True
# # config.model["lstm_cell_size"] = 2048
# config.model["lstm_cell_size"] = 4096
# config.model["max_seq_len"] = 200
# config.model["lstm_use_prev_action"] = True

algo = config.build()



2024-09-26 08:25:46,677	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.resources(num_cpus_per_worker)` has been deprecated. Use `AlgorithmConfig.env_runners(num_cpus_per_env_runner)` instead. This will raise an error in the future!
2024-09-26 08:25:46,677	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.resources(num_gpus_per_worker)` has been deprecated. Use `AlgorithmConfig.env_runners(num_gpus_per_env_runner)` instead. This will raise an error in the future!
2024-09-26 08:25:46,677	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-09-26 08:25:46,677	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\rll

혹시 이전 학습 결과를 로드할 경우

In [5]:
# algo = Algorithm.from_checkpoint("./agents/ModelV2_512x5_2048_v0924_2")

학습 파라미터 재조정

In [5]:
# algo.get_config().training().num_sgd_iter

#Env runner 파라미터 보기.
# algo.env_runner.config["exploration_config"]
# algo.get_config().model

# algo.compute_single_action()
algo.get_policy().model

SACTorchModel(
  (action_model): FullyConnectedNetwork(
    (_logits): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=512, out_features=28, bias=True)
      )
    )
    (_hidden_layers): Sequential(
      (0): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=80, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (1): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (2): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (3): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (4): SlimFC(
        (_model): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
    

학습 시작

In [6]:
from pprint import pprint

n_iter = 4000
save_iter = 0
save_name = "SAC_512x5_GD"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}th iteration done")
    # result.pop("config")
    # pprint(result)

    if i%60 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

algo.save(save_name+str("_final"))

2024-09-26 08:28:33,188	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.


000th iteration done
Checkpoint saved in directory TrainingResult(checkpoint=Checkpoint(filesystem=local, path=SAC_512x5_GD_0), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {}, 'num_env_steps_sampled': 516, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 516, 'num_agent_steps_trained': 0}, 'env_runners': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_timesteps_total': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 0, 'episode_return_max': nan, 'episode_return_min': nan, 'episode_return_mean': nan, 'episodes_this_iter': 0}, 'num_healthy_workers': 12, 'num_in_flight_async_sample_reqs': 0, 'num_remote_worker_restarts': 0, 'num_agent_steps_sampled': 516, 'num_agent_st

2024-09-26 08:28:38,275	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


004th iteration done
005th iteration done
006th iteration done
007th iteration done
008th iteration done
009th iteration done
010th iteration done
011th iteration done
012th iteration done
013th iteration done
014th iteration done
015th iteration done
016th iteration done
017th iteration done
018th iteration done
019th iteration done
020th iteration done
021th iteration done
022th iteration done
023th iteration done
024th iteration done
025th iteration done
026th iteration done
027th iteration done
028th iteration done
029th iteration done
030th iteration done
031th iteration done
032th iteration done
033th iteration done
034th iteration done
035th iteration done
036th iteration done
037th iteration done
038th iteration done
039th iteration done
040th iteration done
041th iteration done
042th iteration done
043th iteration done
044th iteration done
045th iteration done
046th iteration done
047th iteration done
048th iteration done
049th iteration done
050th iteration done
051th iterati

2024-09-26 08:49:24,025	WARNING env_runner_v2.py:158 -- More than 5533 observations in 5533 env steps for episode 915761422727538622 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.


699th iteration done
700th iteration done
701th iteration done
702th iteration done
703th iteration done
704th iteration done
705th iteration done
706th iteration done
707th iteration done
708th iteration done
709th iteration done
710th iteration done
711th iteration done
712th iteration done
713th iteration done
714th iteration done
715th iteration done
716th iteration done
717th iteration done
718th iteration done
719th iteration done
720th iteration done
Checkpoint saved in directory TrainingResult(checkpoint=Checkpoint(filesystem=local, path=SAC_512x5_GD_12), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'actor_loss': -18.769168853759766, 'critic_loss': 0.04710834473371506, 'alpha_loss': -45.43398666381836, 'alpha_value': 0.14483184, 'log_alpha_value': -1.932182, 'target_entropy': -14.0, 'policy_t': 0.012610589154064655, 'mean_q': 17.57681655883789, 'max_q': 20.310266494750977, 'min_q': 15.7811737

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=SAC_512x5_GD_final), metrics={'evaluation': {'env_runners': {'episode_reward_max': -3.0037999516687726, 'episode_reward_min': -10.063848464533008, 'episode_reward_mean': -6.711041139639232, 'episode_len_mean': 6000.0, 'episode_media': {}, 'episodes_timesteps_total': 60000, 'policy_reward_min': {'default_policy': -10.063848464533008}, 'policy_reward_max': {'default_policy': -3.0037999516687726}, 'policy_reward_mean': {'default_policy': -6.711041139639232}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-6.828020145453394, -9.404225547603069, -10.063848464533008, -6.099093019487909, -6.118220504580772, -3.0037999516687726, -7.283216733798697, -4.102921288903254, -7.945020623699355, -6.262045116664098], 'episode_lengths': [6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000], 'policy_default_policy_reward': [-6.828020145453394, -9.404225547603069, -10.063848464533008, -6.099093019487909, -6.118220504580772, -3.003

환경에서 학습된 Policy 테스트하기 (RL Module 사용)

In [ ]:
from ray.rllib.core.rl_module import RLModule
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
import time

algo = Algorithm.from_checkpoint("./ModelV2_512x5_11")


In [ ]:
# env = gym.make("horcrux_terrain_v1/plane-v1", 
#                terminate_when_unhealthy = False, 
#                render_mode = "human", 
#             #    render_camera_name = 'ceiling', 
#                use_gait = True,
#                gait_params = (30,30,40,40,0),
#                **env_config,
#                ) 

# obs, info = env.reset()

# algo.get_policy().action_connectors

In [ ]:
# env = gym.make("horcrux_terrain_v1/sand-v1", 
#                terminate_when_unhealthy = False, 
#                render_mode = "human", 
#             #    render_camera_name = 'ceiling', 
#                use_gait = True,
#                gait_params = (30,30,40,40,0),
#                **env_config,
#                ) 

env = gym.make("horcrux_terrain_v1/plane-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 

for j in range(5):
   episode_return = 0
   terminated = truncated = False

   obs, info = env.reset()


   for i in range(6000):
      action = algo.compute_single_action(obs, explore=False)
      
      obs, reward, terminated, truncated, info = env.step(action)
      
      prev_a = action

      if terminated:
         print("terminated")

      episode_return += reward

   print(f"Reached episode return of {episode_return}.")

env.close()

환경에서 학습된 Policy 테스트하기 (PPO 알고리즘 사용)

In [ ]:
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
from ray.rllib.algorithms.algorithm import Algorithm
import time

env = gym.make("horcrux_terrain_v1/plane-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 

for j in range(10):
   episode_return = 0
   terminated = truncated = False

   obs, info = env.reset()

   init_state = state = [np.zeros([4096], np.float32) for _ in range(200)]
   prev_action = np.zeros((14), np.float32)
   for i in range(1000):

      a, init_state = algo.compute_single_action(observation= obs, state=init_state, prev_action=prev_action,policy_id="default_policy")
      
      obs, reward, terminated, truncated, info = env.step(action)

      prev_action = a
      if terminated:
         print("terminated")

      episode_return += reward

   print(f"Reached episode return of {episode_return}.")

env.close()